In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from google.colab import drive
import re

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Extraction

In [4]:
# Extract text from artical
def extract_article(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch the URL: {url}")
        return None, None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title
    title = soup.find('h1').get_text()

    # Extract the article text
    article_body = soup.find('div', class_='td-post-content tagdiv-type')
    if not article_body:
        print(f"Failed to find the article body for URL: {url}")
        return title, None

    article_text = article_body.get_text(separator='\n')

    return title, article_text

# save artical
def save_article(url_id, title, text):
    if not title or not text:
        print(f"Skipping URL_ID: {url_id} due to missing title or text")
        return

    filename = f"{url_id}.txt"
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(title + '\n\n' + text)

In [5]:
# Read the Excel file
input_file = '/content/drive/MyDrive/NLP Project/NLP Test Assignment/Input.xlsx'
df = pd.read_excel(input_file)

# Process each URL
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    print(f"Processing URL_ID: {url_id}, URL: {url}")

    title, text = extract_article(url)
    save_article(url_id, title, text)


Processing URL_ID: bctech2011, URL: https://insights.blackcoffer.com/ml-and-ai-based-insurance-premium-model-to-predict-premium-to-be-charged-by-the-insurance-company/
Processing URL_ID: bctech2012, URL: https://insights.blackcoffer.com/streamlined-integration-interactive-brokers-api-with-python-for-desktop-trading-application/
Processing URL_ID: bctech2013, URL: https://insights.blackcoffer.com/efficient-data-integration-and-user-friendly-interface-development-navigating-challenges-in-web-application-deployment/
Processing URL_ID: bctech2014, URL: https://insights.blackcoffer.com/effective-management-of-social-media-data-extraction-strategies-for-authentication-security-and-reliability/
Processing URL_ID: bctech2015, URL: https://insights.blackcoffer.com/streamlined-trading-operations-interface-for-metatrader-4-empowering-efficient-management-and-monitoring/
Processing URL_ID: bctech2016, URL: https://insights.blackcoffer.com/efficient-aws-infrastructure-setup-and-management-addressin

In [16]:
# we have put all saved extracted file in a folder name Extracted_text in project folder.

# Text Analysis using NLP

In [6]:
# Load stop words
def load_stop_words(stopwords_dir):
    stopwords = set()
    for filename in os.listdir(stopwords_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(stopwords_dir, filename), 'r', encoding='iso-8859-1') as file:
                for line in file:
                    stopwords.add(line.strip().lower())
    return stopwords

In [18]:
# Path to the StopWords folder
stopwords_dir = '/content/drive/MyDrive/NLP Project/NLP Test Assignment/StopWords'
stopwords = load_stop_words(stopwords_dir)

In [7]:
# load master dictionary
def load_master_dictionary(master_dict_dir):
    positive_words = set()
    negative_words = set()

    with open(os.path.join(master_dict_dir, 'positive-words.txt'), 'r', encoding='iso-8859-1') as file:
        for line in file:
            if line.strip() and not line.startswith(';'):
                positive_words.add(line.strip().lower())

    with open(os.path.join(master_dict_dir, 'negative-words.txt'), 'r', encoding='iso-8859-1') as file:
        for line in file:
            if line.strip() and not line.startswith(';'):
                negative_words.add(line.strip().lower())

    return positive_words, negative_words

In [19]:
master_dict_dir = '/content/drive/MyDrive/NLP Project/NLP Test Assignment/MasterDictionary'
positive_words, negative_words = load_master_dictionary(master_dict_dir)

In [8]:
def clean_text(text, stopwords):
    tokens = word_tokenize(text)
    cleaned_tokens = [token.lower() for token in tokens if token.lower() not in stopwords and token.isalpha()]
    return cleaned_tokens

In [9]:
def calculate_scores(tokens, positive_words, negative_words):
    positive_score = sum(1 for token in tokens if token in positive_words)
    negative_score = sum(1 for token in tokens if token in negative_words)

    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

    return positive_score, negative_score, polarity_score, subjectivity_score

In [10]:
def syllable_count(word):
    word = word.lower()
    count = len(re.findall(r'[aeiouy]', word))
    if word.endswith(('es', 'ed')) and len(word) > 2:
        count -= 1
    return count if count > 0 else 1

In [11]:
def analyze_readability(text):
    sentences = sent_tokenize(text)
    words = [word for word in word_tokenize(text) if word.isalpha()]

    avg_sentence_length = len(words) / len(sentences)
    complex_word_count = sum(1 for word in words if syllable_count(word) > 2)
    percentage_complex_words = complex_word_count / len(words)
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    return avg_sentence_length, percentage_complex_words, fog_index, complex_word_count, len(words)

In [12]:
def count_personal_pronouns(text):
    pronouns = re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE)
    return len(pronouns)


In [13]:
def average_word_length(text):
    words = [word for word in word_tokenize(text) if word.isalpha()]
    return sum(len(word) for word in words) / len(words)

In [15]:
# Output DataFrame
output_columns = [
    'URL_ID','URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
    'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
    'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'
]
output_df = pd.DataFrame(columns=output_columns)

In [17]:
# Set project folder path
project_folder = '/content/drive/MyDrive/NLP Project/NLP Test Assignment'
extracted_text_dir = os.path.join(project_folder, 'Extracted_text')

In [20]:
# Process each URL
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    text_path = os.path.join(extracted_text_dir, f"{url_id}.txt")
    with open(text_path, 'r', encoding='utf-8', errors='ignore') as file:
        text_content = file.read()

    cleaned_tokens = clean_text(text_content, stopwords)
    positive_score, negative_score, polarity_score, subjectivity_score = calculate_scores(cleaned_tokens, positive_words, negative_words)

    avg_sentence_length, perc_complex_words, fog_index, complex_word_count, word_count = analyze_readability(text_content)
    personal_pronouns = count_personal_pronouns(text_content)
    avg_word_length = average_word_length(text_content)

    new_row = pd.DataFrame({
            'URL_ID': [url_id],
            'URL':[url],
            'Positive Score': [positive_score],
            'Negative Score': [negative_score],
            'Polarity Score': [polarity_score],
            'Subjectivity Score': [subjectivity_score],
            'Average Sentence Length': [avg_sentence_length],
            'Percentage of Complex Words': [perc_complex_words],
            'Fog Index': [fog_index],
            'Complex Word Count': [complex_word_count],
            'Word Count': [word_count],
            'Syllable Count': [sum(syllable_count(word) for word in cleaned_tokens)],
            'Personal Pronouns': [personal_pronouns],
            'Average Word Length': [avg_word_length]
        })

    # Concatenate the new row to the output DataFrame
    output_df = pd.concat([output_df, new_row], ignore_index=True)

# Save the output DataFrame to an Excel file
# determining the name of the file
file_name = 'Output Data Structure.xlsx'

# saving the excel
output_df.to_excel(file_name)